## Angular BAO measurements

This started as the script to recreate the BOSS Hubble diagram from the paper: https://arxiv.org/pdf/1607.03155.pdf. I needed it for my slides for the MSSL Seminar and JPL Colloquium (January/February 2018). The notebook had the same name as this one, but without the "Angular". As inputs I was using Florian Beutler's data on D_V/D_V_Planck ratios from his own cool plotting script in C. I had checked the distance calculations against the online Cosmology Calculator: http://www.astro.ucla.edu/~wright/CosmoCalc.html. The Planck best fit was simply from the Wikipedia table here: https://en.wikipedia.org/wiki/Lambda-CDM_model#Parameters.

Now I need a similar plot for the ADAP 2024 ROSES proposals for detecting the BAO in WISE photometric data. For this I only expect to get the D_A, not the fill D_V, so I am making a new clean notebook for it.

From Yun-Ting, survey properties from WISE-PS1 paper I need for the forecast:

But note!! "scale all those dN_ddeg2 values by 43/102.63", actually "118.5/102.63"!

```
===================== BLUE LINE
band 0: sigma_z=None, dN_ddeg2=677.24, z_avg=0.16, bias=1.00
band 1: sigma_z=None, dN_ddeg2=1190.69, z_avg=0.38, bias=1.26
band 2: sigma_z=None, dN_ddeg2=1685.44, z_avg=0.63, bias=1.55
band 3: sigma_z=None, dN_ddeg2=1278.51, z_avg=0.85, bias=1.82
===================== ORANGE LINE
band 0: sigma_z=0.05, dN_ddeg2=548.09, z_avg=0.16, bias=1.00
band 1: sigma_z=0.05, dN_ddeg2=684.54, z_avg=0.38, bias=1.25
band 2: sigma_z=0.05, dN_ddeg2=544.07, z_avg=0.61, bias=1.53
band 3: sigma_z=0.05, dN_ddeg2=162.10, z_avg=0.83, bias=1.79
===================== GREEN LINE
band 0: sigma_z=0.03, dN_ddeg2=427.26, z_avg=0.16, bias=0.99
band 1: sigma_z=0.03, dN_ddeg2=370.02, z_avg=0.37, bias=1.25
band 2: sigma_z=0.03, dN_ddeg2=221.78, z_avg=0.60, bias=1.52
band 3: sigma_z=0.03, dN_ddeg2=33.69, z_avg=0.83, bias=1.79
===================== RED LINE
band 0: sigma_z=0.02, dN_ddeg2=262.98, z_avg=0.15, bias=0.98
band 1: sigma_z=0.02, dN_ddeg2=164.63, z_avg=0.36, bias=1.24
band 2: sigma_z=0.02, dN_ddeg2=60.60, z_avg=0.59, bias=1.51
band 3: sigma_z=0.02, dN_ddeg2=13.28, z_avg=0.87, bias=1.85
=====================
```
 each block correspond to redshift error < sigma_z, i.e., the 4 curves in that dN/dz/ddeg2 plot
z binedges: z = 0-0.25-0.5-0.75-1
https://jpl.slack.com/archives/C06U9PBHYQH/p1714757192874109


<div>
<img src="in/YunTing-Slack-post.png" width="1000"/>
</div>

In [ ]:
import camb
import numpy as np
import scipy.integrate
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import patches
import gcfishlib as gfl

In [ ]:
# Detecting the BAO along the l-o-s
# See Weinberg review, https://arxiv.org/pdf/1201.2434
# p. 63, section 4.4.4
dz_lim = 1000./gfl.c
print('You can see BAO along the line-of-sight if your redshift error < %0.3f.'%dz_lim)

In [ ]:
# https://physics.stackexchange.com/questions/189689/why-is-the-baryon-acoustic-oscillations-bao-calculated-like-this
from astropy.cosmology import Planck18        # Use a Planck 2016 cosmology
from astropy import units as u
d_com = 148 * u.Mpc                           # Comoving size of the BAOs
for z in [0.1, 0.5, 1., 1.5, 2., 3., 4., 100., 3000.]:
    dA    = Planck18.angular_diameter_distance(z) # Angular diameter distance
    d_phy = d_com / (1+z)                         # Physical size of BAO at z
    theta = d_phy / dA * u.rad                    # Angle covered by BAO in radians
    print('BAO size in degrees at z = %0.1f: %0.4s deg '%(z,theta.to(u.deg)))
    print('  Lookback time is '+str(Planck18.lookback_time(z))+'!')

# z = 0.11 it's ~19 deg
# https://www.aanda.org/articles/aa/full_html/2021/05/aa39936-20/aa39936-20.html

# Total comoving volume of WISE
WISE_fsky = 0.5
WISE_vol = WISE_fsky*(Planck18.comoving_volume(1.)-Planck18.comoving_volume(0.1)).to('Gpc3')
print('Total comoving volume between z=0.1-1 over half the sky is '+str(WISE_vol)+'.')

Euclid_spec_fsky = 0.25
Euclid_spec_vol = Euclid_spec_fsky*(Planck18.comoving_volume(1.7)-Planck18.comoving_volume(0.8)).to('Gpc3')
print('Total Euclid comoving volume between z=0.8-1.7 over 1/4 the sky is '+str(Euclid_spec_vol)+'.')

In [ ]:
# Sound horizon pre-calculation and check against Sesh Nadathur's plots (private)
plt.figure(figsize=(10,4))
z_tmp = np.arange(0.1,3.0,0.1)

# Activate CAMB: DESI LCDM is Table 3 here: https://arxiv.org/pdf/2404.03002
H0_DESI = 68.53 ; h_DESI = H0_DESI/100. # DESI + BBN prior on Omega_b
ombh2_DESI = 0.02196# eqn 2.9 here: https://arxiv.org/pdf/2404.03002
omch2_DESI = 0.295*h_DESI**2 - ombh2_DESI
pars = camb.set_params(H0=H0_DESI, ombh2=ombh2_DESI, omch2=omch2_DESI, mnu=0.06, omk=0.) # DESI LCDM
results = camb.get_results(pars)
BAO = results.get_BAO(z_tmp,pars)
#print(BAO[0,:].shape)
rs_DV_ref, H, DA, F_AP = BAO[:,0],BAO[:,1],BAO[:,2],BAO[:,3]
DV = (gfl.c*z_tmp*(1.+z_tmp)**2 * DA**2 / H )**(1./3.)
rd = rs_DV_ref*DV # Sound horizon at drag epoch
if np.abs(np.mean(np.diff(rd))/np.mean(rd))>1.e16: raise Exception('Something is wrong with the sound horizon.')
rd = np.mean(rd)
print('rd in DESI LCDM is',rd,results.sound_horizon(z))
plt.plot(z_tmp,rs_DV_ref**0.,'k-',label='DESI LCDM')

# Planck LCDM
pars_pl = camb.set_params(H0=gfl.H0, ombh2=gfl.Omega_b*gfl.h**2, omch2=gfl.Omega_dm*gfl.h**2, mnu=0.06, omk=1.-gfl.Omega_de-gfl.Omega_dm-gfl.Omega_b) # Planck
results_pl = camb.get_results(pars_pl)
BAO = results_pl.get_BAO(z_tmp,pars_pl)
rs_DV, H, DA, F_AP = BAO[:,0],BAO[:,1],BAO[:,2],BAO[:,3]
DV = (gfl.c*z_tmp*(1.+z_tmp)**2 * DA**2 / H )**(1./3.)
rd = rs_DV*DV
if np.abs(np.mean(np.diff(rd))/np.mean(rd))>1.e16: raise Exception('Something is wrong with the sound horizon.')
rd = np.mean(rd)
print('rd in PLanck LCDM is',rd,results_pl.sound_horizon(z))
plt.plot(z_tmp,(rs_DV_ref/rs_DV),'k--',label='Planck LCDM') # ~factor 1.03 off on top

# Quick plot to cross-check with Sesh (https://euclidconsortium.slack.com/archives/D05HTSS1NJ2/p1714650031791779)
# w0 = −0.55 /+0.39 /−0.21, wa < −1.32 - DESI BAO (eqn 5.4)
H0_DESI = 65.0; h_DESI = H0_DESI/100. # DESI + BBN prior on Omega_b for w0waCDM
ombh2_DESI = 0.02196# eqn 2.9 here: https://arxiv.org/pdf/2404.03002
omch2_DESI = 0.338*h_DESI**2 - ombh2_DESI
pars_dde1 = camb.set_params(H0=H0_DESI, ombh2=ombh2_DESI, omch2=omch2_DESI, mnu=0.06, omk=0.) # DESI LCDM - DE later
pars_dde1.set_dark_energy(w=-0.55, wa=-1.32, dark_energy_model='ppf')
results_dde1 = camb.get_results(pars_dde1)
BAO = results_dde1.get_BAO(z_tmp,pars_dde1)
rs_DV, H, DA, F_AP = BAO[:,0],BAO[:,1],BAO[:,2],BAO[:,3]
DV = (gfl.c*z_tmp*(1.+z_tmp)**2 * DA**2 / H )**(1./3.)
rd = rs_DV*DV
if np.abs(np.mean(np.diff(rd))/np.mean(rd))>1.e16: raise Exception('Something is wrong with the sound horizon.')
rd = np.mean(rd)
print('rd in DDE 1 is',rd,results_dde1.sound_horizon(z))
plt.plot(z_tmp,(rs_DV_ref/rs_DV),'b-.',label='DESI DDE') # ~factor 1.03 off on top

# w0 = −0.727 ± 0.067, wa = −1.05 /+0.31 /−0.27 DESI+CMB+DESY5 (eqn 5.8)
H0_DESI = 67.24; h_DESI = H0_DESI/100. # DESI + BBN prior on Omega_b for w0waCDM
ombh2_DESI = 0.02196# eqn 2.9 here: https://arxiv.org/pdf/2404.03002
omch2_DESI = 0.3160*h_DESI**2 - ombh2_DESI
pars_dde = camb.set_params(H0=H0_DESI, ombh2=ombh2_DESI, omch2=omch2_DESI, mnu=0.06, omk=0.) # DESI LCDM - DE later
pars_dde.set_dark_energy(w=-0.727, wa=-1.05, dark_energy_model='ppf')
results_dde = camb.get_results(pars_dde)
BAO = results_dde.get_BAO(z_tmp,pars_dde)
rs_DV, H, DA, F_AP = BAO[:,0],BAO[:,1],BAO[:,2],BAO[:,3]
DV = (gfl.c*z_tmp*(1.+z_tmp)**2 * DA**2 / H )**(1./3.)
rd = rs_DV*DV
if np.abs(np.mean(np.diff(rd))/np.mean(rd))>1.e16: raise Exception('Something is wrong with the sound horizon.')
rd = np.mean(rd)
print('rd in DDE 2 is',rd,results_dde.sound_horizon(z))
plt.plot(z_tmp,(rs_DV_ref/rs_DV),'-.',color='orange',label='DESI+CMB+DES') # ~factor 1.008 off on top

plt.xlabel('z')
plt.ylabel('(DV/rd)/(DV/rd)_best')
plt.ylim([0.96,1.04])
plt.xlim([0.1,2.5])
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

del rs_DV,rs_DV_ref, H, DA, F_AP, pars, pars_dde, pars_dde1, BAO, results, results_dde1, results_dde, pars_pl, results_pl

# Conversion function to DA
def DMrs_to_DA(z,DMrs,sig): # in principle rd shouldn't depend much on cosmology
    #rs = results.sound_horizon(z)
    rd_planck = gfl.get_rd_Planck()
    return z, DMrs*rd_planck/(1.+z), sig*rd_planck/(1.+z) # NOTE THAT RD_PLANCK IS AN APPROXIMATION!
def DA_to_DMrs(z,DA,sig): # in principle rd shouldn't depend much on cosmology
    rd_planck = gfl.get_rd_Planck()
    return z, DA*(1.+z)/rd_planck, sig*(1.+z)/rd_planck # NOTE THAT RD_PLANCK IS AN APPROXIMATION!


In [ ]:
# Define an object that contains all the properties needed for each survey
class Experiment():
    def __init__(self,
                 name, # string
                 z_vec, # list of redshifts
                 sigDA_vec=None, # set to 0 or None if this is a theoretical model
                 DA_vec=None, # set to None if this is a forecast, ignored if it's a theoretical model
                 DA_is_ratio=False, # if you are providing DA_vec as ratios to DA_Planck, se to true and they will be converted to DA in Mpc(??)
                 color='k',
                 style='-',
                 alpha=1,
                 nolegend=False,
                 binedges=None,
                 xdisplace=0.,
                 **plotpars):
        
        # Set:
        self.name = name
        self.z_vec = np.array(z_vec)
        self.sigDA_vec = np.array(sigDA_vec)
        self.DA_vec = np.array(DA_vec)
        self.plotpars = plotpars
        self.nolegend = nolegend
        self.binedges = binedges
        self.xdisplace = xdisplace
        if DA_is_ratio: fac = np.array(gfl.get_DAPlanck(self.z_vec))
        else: fac = self.z_vec**0.
        
        # With defaults:
        self.color = color
        self.style = style
        self.alpha = alpha
        self.isforecast = False
        self.ismodel = False

        # Derived:
        if sigDA_vec == None or sigDA_vec==0 or 0 in sigDA_vec:
            print(self.name+' is a theory model')
            self.ismodel = True
            self.DA_vec = gfl.get_DAPlanck(self.z_vec)
            self.sigDA_vec = self.z_vec*0. # set all to 0 for modeling
        elif DA_vec == None:
            print(self.name+' is a forecast')
            self.isforecast = True
            self.DA_vec = gfl.get_DAPlanck(self.z_vec)/fac
        else:
            print(self.name+' is a measuremenet')

        # If want to convert from ratio
        self.DA_vec*=fac
        self.sigDA_vec*=fac
        
    # Overwrite the "measured" values with a different cosmology
    def set_nonplanck_cosmology(self,
                                w0=gfl.w0,wa=gfl.wa,
                                H0=gfl.H0, ombh2=gfl.Omega_b*gfl.h**2, omch2=gfl.Omega_dm*gfl.h**2, mnu=0.06, omk=1.-gfl.Omega_de-gfl.Omega_dm,
                                model='ppf',gcfish=True):
        if not self.ismodel and not self.isforecast: raise Exception("You can't overwrite a measurement!")
        if gcfish:
            self.DA_vec = np.array([gfl.D_A(z,gfl.Omega_m,gfl.Omega_de,w0,wa) for z in self.z_vec])
        else:
            pars_dde = camb.set_params(H0,ombh2,omch2,mnu,omk)
            pars_dde.set_dark_energy(w=w0, wa=wa, dark_energy_model=model)
            results_dde = camb.get_results(pars_dde)
            self.DA_vec = results_dde.angular_diameter_distance(self.z_vec)

# And a function to plot them all
def plot_datasets(datasets,
                  alpha=1.,
                  skip=[], 
                  nsig=1.,
                  ratio=False,
                  isDA = False):

    da = []; lab = []
    
    for i, survey in enumerate(datasets):
        
        if survey in skip: continue
        
        if isDA:
            # Plot as a ratio if you provided distances
            if ratio is not False and ratio is not None: DA_Planck = np.array(gfl.get_DAPlanck(survey.z_vec))
            # Do nothing if don't want to plot the ratio or if you already provided a ratio at input
            else: DA_Planck = np.array([1.]*len(survey.z_vec)) 
            
            dist = np.array(survey.DA_vec)/DA_Planck
            ddist = nsig*np.array(survey.sigDA_vec)/DA_Planck
        else:
            print('Converting to DM//rd!')
            # Plot as a ratio if you provided distances
            if ratio is not False and ratio is not None: DMrs_Planck = np.array(gfl.get_DMrsPlanck(survey.z_vec))
            # Do nothing if don't want to plot the ratio or if you already provided a ratio at input
            else: DMrs_Planck = np.array([1.]*len(survey.z_vec)) 
            
            _, survey.DMrs, survey.sigDMrs_vec = DA_to_DMrs(survey.z_vec,survey.DA_vec,survey.sigDA_vec)
            dist = np.array(survey.DMrs)/DMrs_Planck
            ddist = nsig*np.array(survey.sigDMrs_vec)/DMrs_Planck

        # Then overplot the actual error bars with another
        plotpars = {'capsize':10., 'markersize':15,'elinewidth':4.,'barsabove':True, 'linewidth':0., 'capthick':4.}
        for key,value in survey.plotpars.items(): 
            if key in plotpars: plotpars[key] = value
        if survey.ismodel:
            d, = plt.plot(survey.z_vec,dist,color=survey.color, alpha=survey.alpha, linestyle=survey.style,**survey.plotpars,
                          zorder=150+i)
        elif survey.isforecast:
            if survey.binedges is None: 
                #d = plt.fill_between(survey.z_vec, dist-ddist, dist+ddist, color=survey.color, alpha=survey.alpha, **survey.plotpars,
                #                     zorder=i)
                d, _, _ = plt.errorbar(survey.z_vec+survey.xdisplace, dist, xerr=None, yerr=ddist, c=survey.color, alpha=survey.alpha,
                                fmt=survey.style, zorder=i, **plotpars)    
            else:
                for i,x in enumerate(survey.z_vec):
                    ymin = dist[i]-ddist[i]
                    ymax = dist[i]+ddist[i]
                    plt.plot(x,dist[i],survey.style,color=survey.color,alpha=survey.alpha,markeredgecolor='k',
                             zorder = 100+i,label=survey)
                    d = plt.fill_between(np.array([survey.binedges[i],survey.binedges[i+1]]), np.array([ymin, ymin]), np.array([ymax, ymax]), 
                                         color=survey.color, alpha=survey.alpha, zorder = i, **survey.plotpars)

        else:
            d, _, _ = plt.errorbar(survey.z_vec, dist, xerr=None, yerr=ddist, c=survey.color, alpha=survey.alpha,
                                fmt=survey.style, zorder=50+i, **plotpars)

        if survey.nolegend:
            lab.append(None)
        else:
            lab.append(survey.name)
        da.append(d)
    return da, lab

# Plotting settings
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 28}
mpl.rc('font', **font)
#print(plt.style.available)
#[u'seaborn-darkgrid', u'seaborn-notebook', u'classic', u'seaborn-ticks', u'grayscale', 
# u'bmh', u'seaborn-talk', u'dark_background', u'ggplot', u'fivethirtyeight', u'seaborn-colorblind', 
# u'seaborn-deep', u'seaborn-whitegrid', u'seaborn-bright', u'seaborn-poster', u'seaborn-muted', 
# u'seaborn-paper', u'seaborn-white', u'seaborn-pastel', u'seaborn-dark', u'seaborn-dark-palette']
#plt.style.use('seaborn-notebook')
#print mpl.rcParams.keys()
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10

In [ ]:
# Get all the numbers from papers for other datasets and multiply with r_s

# Data & forecasts
experiments = []

# (DECaLS) DR - DM/rd = 18.94 ± 0.61 at zeffective ∼ 0.75 photometry on 4974 deg2 
z, DA, sig = DMrs_to_DA(0.75,18.94,0.61)
print('DECaLS',z,DA,sig)
experiments.append(Experiment("PRLS", [z], [sig], [DA], DA_is_ratio=False, color='k', style='o'))

# DES Y6 - DM/rd = 19.51 ± 0.41  for zeffective = 0.85 photometry on 4273 deg2.
z, DA, sig = DMrs_to_DA(0.85,19.51,0.41)
print('DES',z,DA,sig)
experiments.append(Experiment("DES Y6", [z], [sig], [DA], DA_is_ratio=False, color='g', style='o'))

# These are from Table 1 here: https://arxiv.org/pdf/2404.03002:

# DESI LRG1 DM/rd = 13.62 ± 0.25 at a zeffective = 0.51 on ? deg2
z1, DA1, sig1 = DMrs_to_DA(0.51,13.62,0.25)
print(sig1/DA1)

# DESI LRG2 DM/rd = 16.85 ± 0.32 at a zeffective = 0.71 spectrocopy on ? deg2
z2, DA2, sig2 = DMrs_to_DA(0.71,16.85,0.32)

# DESI L3E1 DM/rd = 21.71 ± 0.28 at a zeffective = 0.930 spectrocopy on ? deg2
z3, DA3, sig3 = DMrs_to_DA(0.930,21.71,0.28)

# DESI ELG2 DM/rd = 27.79 ± 0.69 at a zeffective = 1.317 spectrocopy on ? deg2
z4, DA4, sig4 = DMrs_to_DA(1.317,27.79,0.69)

# DESI Lya QSO DM/rd = 39.71 ± 0.94 at a zeffective = 2.330 spectrocopy on ? deg2
z5, DA5, sig5 = DMrs_to_DA(2.330,39.71,0.94)

experiments.append(Experiment("DESI DR1", 
                              [z1,z2,z3,z4,z5], 
                              [sig1,sig2,sig3,sig4,sig5], 
                              [DA1,DA2,DA3,DA4,DA5], 
                              DA_is_ratio=False, color='b', style='o'))
DA1_ratio = DA1 / gfl.get_DAPlanck([z1])
experiments.append(Experiment("DESI forecast", 
                              [z1], 
                              [0.00919], 
                              DA1_ratio, 
                              DA_is_ratio=True, color='white', style='o', alpha=0.4, markersize=0, nolegend=True, capsize=0))

print('DESI_Lya_QSO',DA5,sig5)
#print('Planck prediction DA',results.angular_diameter_distance(z5))

In [ ]:
theories = []
# Forecasts

# WISE - ADAP proposal - from GCFish code for spectro with some ass#umptions: https://github.com/didamarkovic/GC-Fish
# Initial conservative guess: [0.6, 1.1, 1.5], [0.0213, 0.0185, 0.0769], 
# Initial optimistic guess: [0.6, 1.1, 1.5], [0.00464, 0.0042, 0.0117]
# 2 bins only, RED: sigDA/DA = 0.0145, 0.0289 & z = 0.23, 0.64
# 4 bins, RED: [0.15, 0.36, 0.59, 0.87], [0.0287, 0.0264, 0.047, 0.169]
# 2 bins only, ORANGE: [0.28, 0.66], [0.0153, 0.0128]
# 4 bins, ORANGE: [0.16, 0.38, 0.61, 0.83], [0.0346, 0.0222, 0.0204, 0.0351]
#wise_binedges = [0.,0.25,0.5,0.75,1.0]
wise_binedges = [0.,0.5,1.0]

theories.append(Experiment(r"WISE $\sigma_z=$2% & $n=12.19$M",
                           [0.23, 0.64], [0.0145, 0.0289], #binedges = wise_binedges,
                           DA_is_ratio=True, color='#cc79a7', style='s', #alpha=0.5,
                           capsize=20.,elinewidth=15.,capthick=14))

theories.append(Experiment(r"WISE $\sigma_z=$5% & $n=46.44$M",
                           [0.28, 0.66], [0.0153, 0.0128], #binedges = wise_binedges,
                           DA_is_ratio=True, color='#009e73', style='s', #alpha=0.5,
                           capsize=20.,elinewidth=15.,capthick=14))

#theories.append(Experiment(r"WISE improved 5% $\rightarrow$ 2%",
#                           [0.28, 0.66], [0.0104, 0.00872],  binedges = wise_binedges,
#                           DA_is_ratio=True, color='#f0e442', alpha=0.5))


# Example of calculating from scratch using GCFish code legacy in gcfishlib.py
marged = True
fishfile = 'in/GC-Fish_linear-248468740_8ea09e8_experiment_photz_WISE_yellow_1perc.txt'; da = 'Da'
fm, parameters, indices, zs = gfl.read_fisher(fishfile, marged)
errors, zs = gfl.DA_from_fm(fm, parameters, indices, zs, da, marged)
theories.append(Experiment(r"WISE: 5%$\rightarrow$1% & $n=46.44$M",
                           zs, errors,  #binedges = wise_binedges,
                           DA_is_ratio=True, color='#f0e442', style='s', #alpha=1.0,
                           capsize=20.,elinewidth=15.,capthick=14))

# Euclid - from GCFish code for spectro: https://github.com/didamarkovic/GC-Fish
theories.append(Experiment("Euclid spectro (2030)",
                           [0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75], 
                           [0.00447, 0.00449, 0.00455, 0.00464, 0.0049, 0.00529, 0.00584, 0.00654, 0.00786], 
                           binedges=[0.90, 1.00, 1.10, 1.20, 1.30, 1.40, 1.50, 1.60, 1.70, 1.80],
                           DA_is_ratio=True, color='#0072b2', alpha=0.5, style=''))

# Used color blindness simulator for colors: https://www.color-blindness.com/coblis-color-blindness-simulator/
# With color blind-friendly pallete from here: https://www.color-hex.com/color-palette/49436

plt.figure(figsize=(14,8))

da, lab = plot_datasets(experiments,isDA=False,ratio=False)
z_tmp = np.arange(0.1,2.5,0.05)

da_tmp, = plt.plot(z_tmp,gfl.get_DMrsPlanck(z_tmp)); da.append(da_tmp); lab.append('Planck'); del da_tmp
da_tmp, = plt.plot(z_tmp,z_tmp**0 * 18.94,'y:'); da.append(da_tmp); lab.append('DeCaLS'); del da_tmp
da_tmp, = plt.plot(z_tmp,z_tmp**0 * 19.51,'y-'); da.append(da_tmp); lab.append('DES'); del da_tmp
legend_data = plt.legend(da, lab, loc='upper left', ncol=2, frameon=False, #bbox_to_anchor=(-0.15, 0.95), 
                         handletextpad=0.5, columnspacing = 0.8, fontsize=23); del da, lab
plt.ylabel(r'$D_M/r_d$')
plt.xlabel('z')
plt.xlim([0.4,1.])
plt.ylim([10,25])
plt.show()
plt.clf()
del z_tmp

In [ ]:
# Theory
z_vec = np.arange(0.1,3.,0.1)

# Planck
th1 = Experiment(r"Planck $\Lambda$CDM",z_vec,style='--')
th1.set_nonplanck_cosmology(w0=gfl.w0,wa=gfl.wa,model='fluid')
theories.append(th1)

# DESI Y1 dynamical DE from https://arxiv.org/pdf/2404.03002 
# w0 = −0.55 /+0.39 /−0.21, wa < −1.32 - DESI BAO (eqn 5.4)
# w0 = −0.45 /+0.34 /−0.21, wa = −1.79 /+0.48 /−1.0 DESI BAO+CMB (eqn 5.5)
# w0 = −0.827 ± 0.063, wa = −0.75 /+0.29 /−0.25 DESI+CMB+PantheonPlus (eqn 5.6)
# w0 = −0.64 ± 0.11, wa = −1.27 /+0.40 /−0.34 DESI+CMB+Union3 (eqn 5.7)
# w0 = −0.727 ± 0.067, wa = −1.05 /+0.31 /−0.27 DESI+CMB+DESY5 (eqn 5.8)
#w0_DESI = -0.55; wa_DESI = -1.32; H0_DESI = 65.0; h_DESI = H0_DESI/100. # DESI + BBN prior on Omega_b for w0waCDM
#ombh2_DESI = 0.02196# eqn 2.9 here: https://arxiv.org/pdf/2404.03002
#omch2_DESI = 0.338*h_DESI**2 - ombh2_DESI
w0_DESI = -0.727; wa_DESI = -1.05
H0_DESI = 67.24; h_DESI = H0_DESI/100. # DESI + BBN prior on Omega_b for w0waCDM
ombh2_DESI = 0.02196# eqn 2.9 here: https://arxiv.org/pdf/2404.03002
omch2_DESI = 0.3160*h_DESI**2 - ombh2_DESI
th2 = Experiment(r"w0waCDM",z_vec, style='-.')
th2.set_nonplanck_cosmology(w0=w0_DESI,wa=wa_DESI,model='ppf',
                            H0=H0_DESI, ombh2=ombh2_DESI, omch2=omch2_DESI, mnu=0.06, omk=0.)


theories.append(th2)

In [ ]:
# Now make the plot
plt.figure(figsize=(15, 10))

da, lab = plot_datasets(experiments+theories,ratio=True)

# Plot data
legend_data = plt.legend(da, lab, loc='lower left', bbox_to_anchor=(-0.15, 0.95), ncol=3, frameon=False, 
                         handletextpad=0.5, columnspacing = 0.8, fontsize=23)
# Assuming r_s is fixed over all cosmologies and cancels out in the ratio together with (1+z)!
plt.xlabel(r'$z$')
plt.ylabel(r'HERE')#$D_\text{M}/r_\text{d}$ compared to Planck prediction')
plt.xlim([0.1,1.6])
plt.ylim([0.93,1.059])

# C.f. fig 15 in https://arxiv.org/pdf/2404.03000
plt.savefig('out/ADAP_BAO_forecast'+str(gfl.get_timestamp())+'.png',dpi=300,bbox_inches='tight',transparent=True)
#plt.title(r'Current and forecast $1\sigma$ error bars', fontsize=32)
plt.show()

Dida Markovic, 25 April, 2024